In [110]:
import pygmsh
import meshio
import pygalmesh
import hoomd
import hoomd.md
import numpy as np
import copy

# Make a basic mesh

In [111]:
points = np.array([[-1.0, 0.1, 0.0],[0.0, 0.0, 0.0], [1.0, 0.1, 0.0]])
Npts=len(points)
cells = [("line", np.array([[0, 1],[1, 2]]))]
mesh=meshio.Mesh(points,cells)
NBonds=len(mesh.cells[0].data)
bondindices = list(range(0,NBonds))
bonds=mesh.cells[0].data

mesh = meshio.Mesh(points, cells)
Outputmesh=copy.deepcopy(mesh)
meshio.write("line.vtk",mesh)

angletriples=[0,1,2]

 ## Initialising the simulation 

In [112]:
hoomd.context.initialize("");

Define the snapshot. We will have a unique bond id for every bond in the system, as they will all have different rest lengths. We also want to make a distinction between surface and bulk bonds.

In [113]:
snapshot = hoomd.data.make_snapshot(N=Npts
                                    ,box=hoomd.data.boxdim(Lx=200, Ly=200,dimensions=2)
                                    ,particle_types=['A']
                                    ,bond_types=[str(i) for i in  bondindices]
                                    ,angle_types=['0']
                                   );

Read in the points, bonds and angles

In [114]:
# points
snapshot.particles.position[:] = mesh.points;
snapshot.particles.typeid[0:Npts]=0
# bonds
snapshot.bonds.resize(NBonds)
snapshot.bonds.group[:] = bonds
snapshot.bonds.typeid[:] =[0,1]
#angle triples
snapshot.angles.resize(1)
snapshot.angles.group[:] = angletriples
snapshot.angles.typeid[:] =0

In [115]:
system=hoomd.init.read_snapshot(snapshot);

notice(2): Group "all" created containing 3 particles


Right, lets define the bond type and parameters for each bond

In [116]:
harmonic = hoomd.md.bond.harmonic();

for i in snapshot.bonds.typeid:
    p1,p2 = snapshot.bonds.group[i]
    
    restlength=np.linalg.norm(mesh.points[p2] - mesh.points[p1])
    harmonic.bond_coeff.set(str(i), k=100, r0=restlength);


angle = hoomd.md.angle.harmonic();
angle.angle_coeff.set('0', k=1, t0=np.pi);

Define the integrator. In this case, a langevin dynamics

In [117]:
hoomd.md.integrate.mode_standard(dt=0.1);
all = hoomd.group.all();
integrator = hoomd.md.integrate.nve(group=all);

Define a callback, which we want run periodically

In [118]:
class WritePositions:
    def __init__(self, system):
        self.system = system;
    def __call__(self, timestep):
        snap = self.system.take_snapshot();
        Outputmesh.points = snap.particles.position
        Outputmesh.write("Data/Test/Bond"+str(timestep)+".vtk")
        
hoomd.analyze.callback(callback=WritePositions(system), period=1);

# Run the simulation

In [119]:
hoomd.run(10000);

** starting run **
Time 00:00:04 | Step 10000 / 10000 | TPS 2435.83 | ETA 00:00:00
Average TPS: 2435.35
---------
** run complete **


In [120]:
OutputSnapshot = system.take_snapshot(bonds=True)

In [121]:
print(system.bonds[1])

typeid       : 1
a            : 1
b            : 2
type         : 1



In [122]:
dir(system.angles)

['__class__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'adata',
 'add',
 'angle_data_iterator',
 'get',
 'get_metadata',
 'metadata_fields',
 'remove']

In [123]:
print(system.angles[0])

tag          : 0
typeid       : 0
a            : 0
b            : 1
c            : 2
type         : 0



In [124]:
print(system.angles)

Angle Data for 1 angles of 1 typeid(s)
